In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import csv
import warnings
warnings.filterwarnings('ignore')

# Load core data (always available)
signups = pd.read_csv('data/signups.csv')

# Load subscriptions with proper CSV parsing (handles embedded JSON in metadata column)
def load_subscriptions_csv(filepath):
    """Load subscriptions.csv handling embedded JSON in metadata column"""
    rows = []
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        headers = next(reader)
        for row in reader:
            if len(row) >= 5:  # At least have key columns
                if len(row) < len(headers):
                    row = row + [''] * (len(headers) - len(row))
                elif len(row) > len(headers):
                    row = row[:len(headers)]
                rows.append(row)
    return pd.DataFrame(rows, columns=headers)

subscriptions = load_subscriptions_csv('data/subscriptions.csv')

# Parse dates
signups['created_at'] = pd.to_datetime(signups['created_at'])
subscriptions['created_at'] = pd.to_datetime(subscriptions['created_at'], errors='coerce')

# Convert company_id to numeric for proper matching
subscriptions['company_id'] = pd.to_numeric(subscriptions['company_id'], errors='coerce')

print(f"=== CORE DATA ===")
print(f"Signups: {len(signups)} companies")
print(f"Subscriptions: {len(subscriptions)} records")
print(f"Signups date range: {signups['created_at'].min()} to {signups['created_at'].max()}")

# Load additional data if available
def load_if_exists(filename, date_cols=None):
    path = f'data/{filename}'
    if os.path.exists(path):
        df = pd.read_csv(path, on_bad_lines='skip')
        if date_cols:
            for col in date_cols:
                if col in df.columns:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"  {filename}: {len(df)} records")
        return df
    else:
        print(f"  {filename}: NOT FOUND (run query to export)")
        return None

print(f"\n=== ADDITIONAL DATA ===")
bots = load_if_exists('bots.csv', ['created_at', 'updated_at'])
credit_wallet = load_if_exists('credit_wallet.csv', ['created_at', 'current_period_start', 'current_period_end'])
wallet_transactions = load_if_exists('wallet_transactions.csv', ['created_at'])
stripe_invoices = load_if_exists('stripe_invoices.csv', ['created_at', 'paid_at', 'period_start', 'period_end'])

template_usage = load_if_exists('template_usage_connect.csv', ['first_event', 'last_event'])

sessions_duration = load_if_exists('sessions_duration.csv')
if sessions_duration is not None:
    sessions_duration.columns = ['company_id', 'total_time_minutes', 'avg_session_minutes', 'session_count']
    sessions_duration['company_id'] = pd.to_numeric(sessions_duration['company_id'], errors='coerce')

nodes_usage = load_if_exists('nodes_usage.csv')
if nodes_usage is not None:
    nodes_usage['company_id'] = pd.to_numeric(nodes_usage['company_id'], errors='coerce')

nodes_usage = load_if_exists('nodes_used.csv')
if nodes_usage is not None:
    nodes_usage['company_id'] = pd.to_numeric(nodes_usage['company_id'], errors='coerce')

FileNotFoundError: [Errno 2] No such file or directory: 'data/signups.csv'

## 1. Signups Overview


In [ ]:
# Plan breakdown
print("=== SIGNUPS BY PLAN ===")
plan_counts = signups['plan'].value_counts()
plan_pct = (plan_counts / len(signups) * 100).round(1)
display(pd.DataFrame({'Count': plan_counts, 'Percentage': plan_pct}))


In [ ]:
# Environment and Production breakdown
print("=== ENVIRONMENT ===")
display(signups['environment'].value_counts())

print("\n=== IN PRODUCTION (flag) ===")
display(signups['in_production'].value_counts())

In [ ]:
# Daily signups trend
signups['date'] = signups['created_at'].dt.date
daily_signups = signups.groupby('date').size().reset_index(name='count')
print(f"Average signups per day: {daily_signups['count'].mean():.1f}")
print(f"Min: {daily_signups['count'].min()}, Max: {daily_signups['count'].max()}")
daily_signups.tail(10)


## 2. Subscriptions Analysis


In [ ]:
# Subscription status breakdown
print("=== SUBSCRIPTION STATUS ===")
display(subscriptions['status'].value_counts())

print("\n=== SUBSCRIPTION BY PRODUCT ===")
product_counts = subscriptions['product_name'].value_counts()
display(product_counts)

# Show the actual product names we're detecting
print("\n=== PRODUCT DETECTION ===")
brain_pattern = subscriptions['product_name'].str.contains('Brain', case=False, na=False)
connect_pattern = subscriptions['product_name'].str.contains('Connect', case=False, na=False)
print(f"Brain-related products: {brain_pattern.sum()} subscriptions")
print(f"Connect-related products: {connect_pattern.sum()} subscriptions")

print("\n=== PRODUCT x STATUS ===")
display(pd.crosstab(subscriptions['product_name'], subscriptions['status'], margins=True))

## 3. Joining Signups with Subscriptions


In [ ]:
# How many signups have subscriptions?
companies_with_subs = subscriptions['company_id'].unique()
signups['has_subscription'] = signups['company_id'].isin(companies_with_subs)

print("=== SIGNUPS WITH SUBSCRIPTIONS ===")
sub_counts = signups['has_subscription'].value_counts()
sub_pct = (sub_counts / len(signups) * 100).round(1)
display(pd.DataFrame({'Count': sub_counts, 'Percentage': sub_pct}))


In [ ]:
# Create subscription summary per company
sub_summary = subscriptions.groupby('company_id').agg({
    'subscription_id': 'count',
    'status': lambda x: list(x.unique()),
    'product_name': lambda x: list(x.unique()),
    'created_at': 'min'
}).reset_index()
sub_summary.columns = ['company_id', 'subscription_count', 'statuses', 'products', 'first_subscription']

# Check for active subscriptions
sub_summary['has_active'] = sub_summary['statuses'].apply(lambda x: 'ACTIVE' in x)
sub_summary['has_trialing'] = sub_summary['statuses'].apply(lambda x: 'TRIALING' in x)
# Check for Brain and Connect products using pattern matching
# This handles: "Brain studio", "Brain conversaciones", "Connect", "Plan Connect"
def has_brain_product(products):
    if not isinstance(products, list):
        return False
    return any('brain' in str(p).lower() for p in products)

def has_connect_product(products):
    if not isinstance(products, list):
        return False
    return any('connect' in str(p).lower() for p in products)

sub_summary['has_brain_studio'] = sub_summary['products'].apply(has_brain_product)
sub_summary['has_connect'] = sub_summary['products'].apply(has_connect_product)

# Track Connect-specific statuses for funnel
connect_subs = subscriptions[subscriptions['product_name'].str.contains('Connect', case=False, na=False)]
connect_active_df = connect_subs[connect_subs['status'] == 'ACTIVE'].groupby('company_id').size().reset_index(name='connect_active_count')
connect_trialing_df = connect_subs[connect_subs['status'] == 'TRIALING'].groupby('company_id').size().reset_index(name='connect_trialing_count')

sub_summary = sub_summary.merge(connect_active_df, on='company_id', how='left')
sub_summary = sub_summary.merge(connect_trialing_df, on='company_id', how='left')
sub_summary['connect_active'] = sub_summary['connect_active_count'].fillna(0) > 0
sub_summary['connect_trialing'] = sub_summary['connect_trialing_count'].fillna(0) > 0

# Merge with signups
analysis = signups.merge(sub_summary, on='company_id', how='left')
analysis['subscription_count'] = analysis['subscription_count'].fillna(0).astype(int)
analysis['has_active'] = analysis['has_active'].fillna(False)
analysis['has_trialing'] = analysis['has_trialing'].fillna(False)
analysis['has_brain_studio'] = analysis['has_brain_studio'].fillna(False)
analysis['has_connect'] = analysis['has_connect'].fillna(False)
analysis['connect_active'] = analysis['connect_active'].fillna(False)
analysis['connect_trialing'] = analysis['connect_trialing'].fillna(False)

print(f"Analysis dataset: {len(analysis)} companies")
print(f"\n=== PRODUCT DETECTION RESULTS ===")
print(f"Companies with Brain Studio: {analysis['has_brain_studio'].sum()}")
print(f"Companies with Connect: {analysis['has_connect'].sum()}")
print(f"  - Connect Active (paid): {analysis['connect_active'].sum()}")
print(f"  - Connect Trialing: {analysis['connect_trialing'].sum()}")
analysis.head()

## 4. Conversion Funnel


In [ ]:
# Build conversion funnel
total = len(analysis)
has_any_sub = analysis['has_subscription'].sum()
has_brain = analysis['has_brain_studio'].sum()
has_connect = analysis['has_connect'].sum()
connect_trialing = analysis['connect_trialing'].sum()
connect_active = analysis['connect_active'].sum()
has_active = analysis['has_active'].sum()
in_production = analysis['in_production'].sum()

print("=== OVERALL FUNNEL ===")
funnel = pd.DataFrame({
    'Stage': ['Total Signups', 'Has Any Subscription', 'Has Brain Studio', 'Has Connect', 'Active Subscription', 'In Production (flag)'],
    'Count': [total, has_any_sub, has_brain, has_connect, has_active, in_production],
})
funnel['Percentage'] = (funnel['Count'] / total * 100).round(1)
funnel['Conversion'] = funnel['Percentage'].astype(str) + '%'
display(funnel)

print("\n=== CONNECT-SPECIFIC FUNNEL ===")
connect_funnel = pd.DataFrame({
    'Stage': ['Total Signups', 'Started Connect (trial or active)', 'Connect Trialing', 'Connect Active (Paid)'],
    'Count': [total, has_connect, connect_trialing, connect_active],
})
connect_funnel['Percentage'] = (connect_funnel['Count'] / total * 100).round(1)
connect_funnel['Conversion'] = connect_funnel['Percentage'].astype(str) + '%'

# Trial to paid conversion
if has_connect > 0:
    trial_to_paid = connect_active / has_connect * 100
    print(f"Trial → Paid Conversion: {trial_to_paid:.1f}%")
display(connect_funnel)

In [ ]:
# Breakdown by plan type
print("=== CONVERSION BY PLAN TYPE ===")
plan_analysis = analysis.groupby('plan').agg({
    'company_id': 'count',
    'has_subscription': 'sum',
    'has_active': 'sum',
    'in_production': 'sum'
}).reset_index()
plan_analysis.columns = ['plan', 'total', 'has_subscription', 'has_active', 'in_production']
plan_analysis['sub_rate'] = (plan_analysis['has_subscription'] / plan_analysis['total'] * 100).round(1)
plan_analysis['active_rate'] = (plan_analysis['has_active'] / plan_analysis['total'] * 100).round(1)
plan_analysis['prod_rate'] = (plan_analysis['in_production'] / plan_analysis['total'] * 100).round(1)
plan_analysis


## 5. Self-Service Deep Dive

Focus on SELF_SERVICE users since those are the ones expected to convert on their own


In [ ]:
# Filter to self-service only
self_service = analysis[analysis['plan'] == 'SELF_SERVICE'].copy()
print(f"Self-service signups: {len(self_service)}")

# Breakdown
ss_total = len(self_service)
ss_with_sub = self_service['has_subscription'].sum()
ss_active = self_service['has_active'].sum()
ss_trialing = self_service['has_trialing'].sum()
ss_in_prod = self_service['in_production'].sum()

print(f"\n=== SELF-SERVICE FUNNEL ===")
print(f"Total: {ss_total}")
print(f"Has subscription: {ss_with_sub} ({ss_with_sub/ss_total*100:.1f}%)")
print(f"  - Active: {ss_active} ({ss_active/ss_total*100:.1f}%)")
print(f"  - Trialing: {ss_trialing} ({ss_trialing/ss_total*100:.1f}%)")
print(f"In Production: {ss_in_prod} ({ss_in_prod/ss_total*100:.1f}%)")

In [ ]:
# Self-service WITHOUT subscription - who are they?
ss_no_sub = self_service[~self_service['has_subscription']]
print(f"Self-service WITHOUT subscription: {len(ss_no_sub)}")
print(f"\nEnvironment breakdown:")
display(ss_no_sub['environment'].value_counts())
print(f"\nIn production:")
display(ss_no_sub['in_production'].value_counts())


## 6. Key Insights Summary


In [ ]:
print("="*60)
print("KEY INSIGHTS")
print("="*60)

print(f"\n1. TOTAL SIGNUPS: {len(signups)}")
print(f"   - Self-Service: {len(signups[signups['plan']=='SELF_SERVICE'])} ({len(signups[signups['plan']=='SELF_SERVICE'])/len(signups)*100:.1f}%)")
print(f"   - Enterprise: {len(signups[signups['plan']=='ENTERPRISE'])} ({len(signups[signups['plan']=='ENTERPRISE'])/len(signups)*100:.1f}%)")

print(f"\n2. SUBSCRIPTION STATUS:")
print(f"   - Companies with ANY subscription: {has_any_sub} ({has_any_sub/total*100:.1f}%)")
print(f"   - Companies with ACTIVE subscription: {has_active} ({has_active/total*100:.1f}%)")

print(f"\n3. PRODUCTION STATUS:")
print(f"   - In Production (flag=1): {in_production} ({in_production/total*100:.1f}%)")
print(f"   - Environment=PRODUCTION: {len(signups[signups['environment']=='PRODUCTION'])} ({len(signups[signups['environment']=='PRODUCTION'])/total*100:.1f}%)")

print(f"\n4. POTENTIAL ISSUES:")
no_sub = total - has_any_sub
print(f"   - Signups WITHOUT any subscription: {no_sub} ({no_sub/total*100:.1f}%)")
print(f"   - These users signed up but never started a trial/subscription")


## 7. CORRECTED Conversion Funnel (Using New Data Sources)

The previous funnel was WRONG because Brain Studio subscriptions are auto-created and FREE.
Now we use the correct data sources:
- `bots.csv` - Shows actual bot creation and production channel connection
- `credit_wallet.csv` - Shows conversation usage and free tier exhaustion
- `stripe_invoices.csv` - Shows actual payments


In [ ]:
# Build CORRECTED funnel using new data sources
# Focus on SELF_SERVICE signups (organic users expected to self-convert)

self_service = signups[signups['plan'] == 'SELF_SERVICE'].copy()
ss_total = len(self_service)
print(f"=== CORRECTED FUNNEL (SELF_SERVICE only) ===")
print(f"Total Self-Service Signups: {ss_total}")

# Stage 1: Created Bot (from bots.csv)
if bots is not None:
    companies_with_bots = bots['company_id'].unique()
    self_service['has_bot'] = self_service['company_id'].isin(companies_with_bots)
    ss_with_bot = self_service['has_bot'].sum()
    print(f"\n1. Created Bot: {ss_with_bot} ({ss_with_bot/ss_total*100:.1f}%)")
    
    # Stage 2: Connected Production Channel (bots.in_production = 1)
    prod_bots = bots[bots['in_production'] == 1]
    companies_with_prod = prod_bots['company_id'].unique()
    self_service['has_prod_channel'] = self_service['company_id'].isin(companies_with_prod)
    ss_with_prod = self_service['has_prod_channel'].sum()
    print(f"2. Production Channel: {ss_with_prod} ({ss_with_prod/ss_total*100:.1f}%)")
else:
    print("\n[!] bots.csv not available - run queries/bots.sql")
    self_service['has_bot'] = False
    self_service['has_prod_channel'] = False

# Stage 3: Used Conversations (from credit_wallet)
if credit_wallet is not None:
    # Companies that have used any conversations
    used_convos = credit_wallet[credit_wallet['total_used'] > 0]
    companies_used = used_convos['company_id'].unique()
    self_service['used_conversations'] = self_service['company_id'].isin(companies_used)
    ss_used = self_service['used_conversations'].sum()
    print(f"3. Used Conversations: {ss_used} ({ss_used/ss_total*100:.1f}%)")
    
    # Stage 4: Exceeded Free Tier
    exceeded = credit_wallet[credit_wallet['exceeded_free_tier'] == 1]
    companies_exceeded = exceeded['company_id'].unique()
    self_service['exceeded_free_tier'] = self_service['company_id'].isin(companies_exceeded)
    ss_exceeded = self_service['exceeded_free_tier'].sum()
    print(f"4. Exceeded Free Tier: {ss_exceeded} ({ss_exceeded/ss_total*100:.1f}%)")
else:
    print("\n[!] credit_wallet.csv not available - run queries/credit_wallet.sql")
    self_service['used_conversations'] = False
    self_service['exceeded_free_tier'] = False

# Stage 5: Actually Paid (from stripe_invoices)
if stripe_invoices is not None:
    paid_invoices = stripe_invoices[stripe_invoices['amount_paid'] > 0]
    companies_paid = paid_invoices['company_id'].unique()
    self_service['actually_paid'] = self_service['company_id'].isin(companies_paid)
    ss_paid = self_service['actually_paid'].sum()
    print(f"5. Actually Paid: {ss_paid} ({ss_paid/ss_total*100:.1f}%)")
    
    # This is the REAL conversion rate!
    print(f"\n>>> REAL CONVERSION RATE: {ss_paid/ss_total*100:.2f}% <<<")
else:
    print("\n[!] stripe_invoices.csv not available - run queries/stripe_invoices.sql")
    self_service['actually_paid'] = False


In [ ]:
# Visualize the corrected funnel
funnel_stages = ['Signup', 'Created Bot', 'Production Channel', 'Used Conversations', 'Exceeded Free', 'Actually Paid']
funnel_values = [ss_total]

if bots is not None:
    funnel_values.append(ss_with_bot)
    funnel_values.append(ss_with_prod)
else:
    funnel_values.extend([0, 0])

if credit_wallet is not None:
    funnel_values.append(ss_used)
    funnel_values.append(ss_exceeded)
else:
    funnel_values.extend([0, 0])

if stripe_invoices is not None:
    funnel_values.append(ss_paid)
else:
    funnel_values.append(0)

corrected_funnel = pd.DataFrame({
    'Stage': funnel_stages,
    'Count': funnel_values,
})
corrected_funnel['Percentage'] = (corrected_funnel['Count'] / ss_total * 100).round(1)
corrected_funnel['Drop-off'] = corrected_funnel['Count'].diff().fillna(0).astype(int)
corrected_funnel['Drop-off %'] = (corrected_funnel['Drop-off'].abs() / corrected_funnel['Count'].shift(1) * 100).fillna(0).round(1)

print("=== CORRECTED FUNNEL TABLE ===")
corrected_funnel


## 8. Deep Dive: Bots Analysis


In [ ]:
if bots is not None:
    print("=== BOTS OVERVIEW ===")
    print(f"Total bots: {len(bots)}")
    print(f"Unique companies with bots: {bots['company_id'].nunique()}")
    
    print("\n=== BOT TYPES ===")
    display(bots['type'].value_counts())
    
    print("\n=== BOT STATES ===")
    display(bots['state'].value_counts())
    
    print("\n=== PRODUCTION STATUS ===")
    display(bots['in_production'].value_counts())
    
    print("\n=== CONNECTED STATUS ===")
    if 'connected' in bots.columns:
        display(bots['connected'].value_counts())
    
    # Bots per company
    bots_per_company = bots.groupby('company_id').size().describe()
    print("\n=== BOTS PER COMPANY ===")
    display(bots_per_company)
else:
    print("bots.csv not available - run queries/bots.sql and export")


## 9. Deep Dive: Credit Wallet & Free Tier


In [ ]:
if credit_wallet is not None:
    print("=== CREDIT WALLET OVERVIEW ===")
    print(f"Total wallets: {len(credit_wallet)}")
    print(f"Unique companies: {credit_wallet['company_id'].nunique()}")
    
    print("\n=== FREE CONVERSATIONS ALLOCATION ===")
    display(credit_wallet['free_conversations'].describe())
    
    print("\n=== TOTAL USED ===")
    display(credit_wallet['total_used'].describe())
    
    print("\n=== EXCEEDED FREE TIER ===")
    display(credit_wallet['exceeded_free_tier'].value_counts())
    
    # How many used vs how many exceeded
    used_any = (credit_wallet['total_used'] > 0).sum()
    exceeded = (credit_wallet['exceeded_free_tier'] == 1).sum()
    print(f"\nCompanies that used ANY conversations: {used_any}")
    print(f"Companies that EXCEEDED free tier: {exceeded}")
    print(f"Conversion from usage to paid tier: {exceeded/used_any*100:.1f}%" if used_any > 0 else "N/A")
    
    # Distribution of usage as % of free tier
    if 'free_tier_usage_pct' in credit_wallet.columns:
        print("\n=== FREE TIER USAGE % DISTRIBUTION ===")
        usage_buckets = pd.cut(credit_wallet['free_tier_usage_pct'], 
                               bins=[0, 25, 50, 75, 100, 150, 200, float('inf')],
                               labels=['0-25%', '25-50%', '50-75%', '75-100%', '100-150%', '150-200%', '>200%'])
        display(usage_buckets.value_counts().sort_index())
else:
    print("credit_wallet.csv not available - run queries/credit_wallet.sql and export")


## 10. Deep Dive: Payments & Revenue


## 11. Save Combined Analysis for Streamlit


In [ ]:
# Build comprehensive analysis dataset with all new data sources
# Start with signups base
final_analysis = signups.copy()

# Add bot info
if bots is not None:
    bot_summary = bots.groupby('company_id').agg({
        'bot_id': 'count',
        'in_production': 'sum',
        'type': lambda x: list(x.unique()),
        'created_at': 'min'
    }).reset_index()
    bot_summary.columns = ['company_id', 'bot_count', 'production_bots', 'bot_types', 'first_bot_created']
    final_analysis = final_analysis.merge(bot_summary, on='company_id', how='left')
    final_analysis['bot_count'] = final_analysis['bot_count'].fillna(0).astype(int)
    final_analysis['production_bots'] = final_analysis['production_bots'].fillna(0).astype(int)
    final_analysis['has_bot'] = final_analysis['bot_count'] > 0
    final_analysis['has_prod_channel'] = final_analysis['production_bots'] > 0

# Add credit wallet info
if credit_wallet is not None:
    wallet_cols = ['company_id', 'total_used', 'free_conversations', 'exceeded_free_tier', 'balance']
    wallet_cols = [c for c in wallet_cols if c in credit_wallet.columns]
    final_analysis = final_analysis.merge(credit_wallet[wallet_cols], on='company_id', how='left')
    final_analysis['total_used'] = final_analysis['total_used'].fillna(0)
    final_analysis['exceeded_free_tier'] = final_analysis['exceeded_free_tier'].fillna(0).astype(int)
    final_analysis['used_conversations'] = final_analysis['total_used'] > 0

# Add payment info from stripe invoices
if stripe_invoices is not None:
    payment_summary = stripe_invoices.groupby('company_id').agg({
        'amount_paid': 'sum',
        'paid_at': 'min'
    }).reset_index()
    payment_summary.columns = ['company_id', 'total_paid', 'first_payment']
    final_analysis = final_analysis.merge(payment_summary, on='company_id', how='left')
    final_analysis['total_paid'] = final_analysis['total_paid'].fillna(0)
    final_analysis['actually_paid'] = final_analysis['total_paid'] > 0

# Save for Streamlit
final_analysis.to_csv('data/analysis_combined.csv', index=False)
print(f"Saved: data/analysis_combined.csv ({len(final_analysis)} companies)")
print(f"\nColumns: {list(final_analysis.columns)}")


In [ ]:
# Save combined analysis for Streamlit
print("=== FINAL ANALYSIS SUMMARY ===")
print(f"Total companies: {len(analysis)}")
print(f"\nProduct subscriptions:")
print(f"  - Brain Studio: {analysis['has_brain_studio'].sum()}")
print(f"  - Connect: {analysis['has_connect'].sum()}")
print(f"    - Trialing: {analysis['connect_trialing'].sum()}")
print(f"    - Active (Paid): {analysis['connect_active'].sum()}")

analysis.to_csv('data/analysis_combined.csv', index=False)
print("\nSaved: data/analysis_combined.csv")
print(f"Columns: {list(analysis.columns)}")


## 12. User Sessions & Retention Analysis

The retention curve needs to be calculated correctly:
- **Current issue**: Using "last_session >= X days after signup" is WRONG
- **Correct approach**: Count companies that logged in during each week period after signup

We need to join `user_sessions.csv` with `signups.csv` to calculate:
1. Days from signup to first session
2. Days from signup to last session  
3. Whether they were active during Week 1, Week 2, Week 3, Week 4


In [ ]:
# Load user sessions data
user_sessions_path = 'data/user_sessions.csv'
if os.path.exists(user_sessions_path):
    user_sessions = pd.read_csv(user_sessions_path)
    print(f"User sessions: {len(user_sessions)} companies")
    print(f"Columns: {list(user_sessions.columns)}")
    
    # Parse dates - handle ISO format with Z suffix
    user_sessions['first_session'] = pd.to_datetime(user_sessions['first_session'], utc=True).dt.tz_convert(None)
    user_sessions['last_session'] = pd.to_datetime(user_sessions['last_session'], utc=True).dt.tz_convert(None)
    
    # Basic stats
    print(f"\n=== SESSION STATS ===")
    print(f"Days active distribution:")
    display(user_sessions['days_active'].describe())
    
    # How many have 7+ days active?
    print(f"\n=== ACTIVITY THRESHOLDS ===")
    print(f"Companies with 1+ days active: {(user_sessions['days_active'] >= 1).sum()}")
    print(f"Companies with 7+ days active: {(user_sessions['days_active'] >= 7).sum()}")
    print(f"Companies with 14+ days active: {(user_sessions['days_active'] >= 14).sum()}")
    print(f"Companies with 28+ days active: {(user_sessions['days_active'] >= 28).sum()}")
else:
    print("user_sessions.csv not found - run MongoDB query to export")

In [ ]:
# Join user_sessions with signups to calculate retention properly
# Filter to SELF_SERVICE and Nov 15+ signups

# Get self-service signups from Nov 15, 2025 onwards
nov15 = pd.Timestamp('2025-11-15')
self_service_signups = signups[
    (signups['plan'] == 'SELF_SERVICE') & 
    (signups['created_at'] >= nov15)
].copy()

print(f"Self-service signups from Nov 15+: {len(self_service_signups)}")

# Merge with user_sessions
user_sessions['company_id'] = pd.to_numeric(user_sessions['company_id'], errors='coerce')
self_service_signups['company_id'] = pd.to_numeric(self_service_signups['company_id'], errors='coerce')

retention_df = self_service_signups.merge(
    user_sessions[['company_id', 'first_session', 'last_session', 'days_active', 'total_sessions']],
    on='company_id',
    how='left'
)

print(f"Companies with session data: {retention_df['first_session'].notna().sum()}")
print(f"Companies WITHOUT session data: {retention_df['first_session'].isna().sum()}")

# Show sample
retention_df[['company_id', 'company_name', 'created_at', 'first_session', 'last_session', 'days_active']].head(10)


In [ ]:
# Calculate CORRECT retention metrics
# 
# Retention = "Of users who signed up X days ago, how many logged in at least once after day Y?"
#
# Key insight: We're measuring "last_session - signup_date" to see how long they stayed active
# But this is STILL imperfect - ideally we'd have daily login data per company

# Calculate days from signup to last session
retention_df['signup_date'] = retention_df['created_at'].dt.normalize()
retention_df['last_active_date'] = retention_df['last_session'].dt.normalize()
retention_df['first_active_date'] = retention_df['first_session'].dt.normalize()

# Days from signup to last activity
retention_df['days_to_last_activity'] = (retention_df['last_active_date'] - retention_df['signup_date']).dt.days

# Days from signup to first activity (how fast did they start?)
retention_df['days_to_first_activity'] = (retention_df['first_active_date'] - retention_df['signup_date']).dt.days

# How old is this signup?
today = pd.Timestamp.now().normalize()
retention_df['days_since_signup'] = (today - retention_df['signup_date']).dt.days

print("=== RETENTION METRICS ===")
print(f"Total signups in cohort: {len(retention_df)}")
print(f"With any session data: {retention_df['days_to_last_activity'].notna().sum()}")

# Show distribution
print("\n=== Days to Last Activity (distribution) ===")
display(retention_df['days_to_last_activity'].describe())

print("\n=== Days to First Activity (how fast they started) ===")
display(retention_df['days_to_first_activity'].describe())


In [ ]:
# CORRECT Retention Calculation
#
# For each period, we need to count:
# 1. ELIGIBLE: signups that are OLD ENOUGH to be measured (signed up at least X days ago)
# 2. RETAINED: of those eligible, how many had their last activity >= X days after signup
#
# This tells us: "Of people who had time to reach week N, how many were still active at week N?"

def calculate_retention(df, period_days, min_signup_age):
    """
    Calculate retention for a specific period.
    
    Args:
        df: DataFrame with 'days_since_signup' and 'days_to_last_activity'
        period_days: The retention period we're measuring (e.g., 7 for week 1)
        min_signup_age: Minimum days since signup to be eligible
    
    Returns:
        dict with eligible count, retained count, and rate
    """
    # Filter to signups that are old enough
    eligible = df[df['days_since_signup'] >= min_signup_age]
    
    # Of those, count how many had activity at or after the period
    # (meaning they were still using the product at that point)
    retained = eligible[eligible['days_to_last_activity'] >= period_days]
    
    eligible_count = len(eligible)
    retained_count = len(retained[retained['days_to_last_activity'].notna()])
    
    rate = retained_count / eligible_count * 100 if eligible_count > 0 else 0
    
    return {
        'eligible': eligible_count,
        'retained': retained_count,
        'rate': rate
    }

# Calculate retention for each period
periods = [
    ('Day 1', 1, 7),      # Logged in at least 1 day after signup, need 7 days of age
    ('Week 1', 7, 14),    # Still active after 7 days, need 14 days of age
    ('Week 2', 14, 21),   # Still active after 14 days, need 21 days of age
    ('Week 3', 21, 28),   # Still active after 21 days, need 28 days of age
    ('Week 4', 28, 35),   # Still active after 28 days, need 35 days of age
    ('Week 8', 56, 63),   # Still active after 56 days, need 63 days of age
]

print("=== CORRECT RETENTION CALCULATION ===")
print("(Based on: last_activity >= period_days after signup)")
print()

retention_results = []
for name, period_days, min_age in periods:
    result = calculate_retention(retention_df, period_days, min_age)
    retention_results.append({
        'period': name,
        'period_days': period_days,
        'eligible': result['eligible'],
        'retained': result['retained'],
        'rate': result['rate']
    })
    print(f"{name}: {result['retained']}/{result['eligible']} = {result['rate']:.1f}%")

retention_curve = pd.DataFrame(retention_results)
display(retention_curve)


In [ ]:
# Let's also look at "days_active" as a quality metric
# This tells us: of all unique days since their first session, how many did they log in?

print("=== ACTIVITY QUALITY: Days Active Distribution ===")
print("(Number of unique days they logged in)")
print()

# Only look at companies with session data
with_sessions = retention_df[retention_df['days_active'].notna()].copy()
print(f"Companies with session data: {len(with_sessions)}")

if len(with_sessions) > 0:
    print(f"\nDays Active Stats:")
    display(with_sessions['days_active'].describe())
    
    # Buckets
    print("\n=== ACTIVITY BUCKETS ===")
    buckets = [
        (1, 1, "1 day only"),
        (2, 3, "2-3 days"),
        (4, 7, "4-7 days (about a week)"),
        (8, 14, "8-14 days (1-2 weeks)"),
        (15, 28, "15-28 days (2-4 weeks)"),
        (29, 56, "29-56 days (1-2 months)"),
        (57, 999, "57+ days (2+ months)")
    ]
    
    for min_d, max_d, label in buckets:
        count = len(with_sessions[(with_sessions['days_active'] >= min_d) & (with_sessions['days_active'] <= max_d)])
        pct = count / len(with_sessions) * 100
        print(f"  {label}: {count} ({pct:.1f}%)")

## 13. Build Final Analysis CSV with Retention Data

Now we'll save a comprehensive `analysis_combined.csv` that includes:
- All signup data
- Subscription info (Brain, Connect)
- Bot data (has_bot, has_prod_channel)
- Payment data (actually_paid, total_paid)
- Session/retention data (days_active, last_session, retention flags)


In [ ]:
# Build comprehensive final analysis
# Start with base signups (filtered to Nov 15+)
nov15 = pd.Timestamp('2025-11-15')
final = signups[signups['created_at'] >= nov15].copy()
print(f"Signups from Nov 15, 2025+: {len(final)}")

# Ensure company_id is numeric
final['company_id'] = pd.to_numeric(final['company_id'], errors='coerce')

# --- Add subscription info ---
if subscriptions is not None:
    subs = subscriptions.copy()
    subs['company_id'] = pd.to_numeric(subs['company_id'], errors='coerce')
    
    # Companies with any subscription
    companies_with_subs = subs['company_id'].dropna().unique()
    final['has_subscription'] = final['company_id'].isin(companies_with_subs)
    
    # Active/Trialing
    active_companies = subs[subs['status'] == 'ACTIVE']['company_id'].unique()
    trialing_companies = subs[subs['status'] == 'TRIALING']['company_id'].unique()
    final['has_active'] = final['company_id'].isin(active_companies)
    final['has_trialing'] = final['company_id'].isin(trialing_companies)
    
    # Brain Studio
    brain_subs = subs[subs['product_name'].str.contains('Brain', case=False, na=False)]
    brain_companies = brain_subs['company_id'].unique()
    final['has_brain_studio'] = final['company_id'].isin(brain_companies)
    brain_active = brain_subs[brain_subs['status'] == 'ACTIVE']['company_id'].unique()
    final['brain_active'] = final['company_id'].isin(brain_active)
    
    # Connect
    connect_subs = subs[subs['product_name'].str.contains('Connect', case=False, na=False)]
    connect_companies = connect_subs['company_id'].unique()
    final['has_connect'] = final['company_id'].isin(connect_companies)
    connect_active = connect_subs[connect_subs['status'] == 'ACTIVE']['company_id'].unique()
    connect_trialing = connect_subs[connect_subs['status'] == 'TRIALING']['company_id'].unique()
    final['connect_active'] = final['company_id'].isin(connect_active)
    final['connect_trialing'] = final['company_id'].isin(connect_trialing)
else:
    for col in ['has_subscription', 'has_active', 'has_trialing', 'has_brain_studio', 'brain_active', 
                'has_connect', 'connect_active', 'connect_trialing']:
        final[col] = False

print(f"Subscription columns added")


In [ ]:
# --- Add bot info ---
if bots is not None:
    bots_copy = bots.copy()
    bots_copy['company_id'] = pd.to_numeric(bots_copy['company_id'], errors='coerce')
    
    bot_companies = bots_copy['company_id'].unique()
    final['has_bot'] = final['company_id'].isin(bot_companies)
    
    # Production channel
    prod_bots = bots_copy[bots_copy['in_production'] == 1]
    prod_companies = prod_bots['company_id'].unique()
    final['has_prod_channel'] = final['company_id'].isin(prod_companies)
    
    # Bot count
    bot_counts = bots_copy.groupby('company_id').size().reset_index(name='bot_count')
    final = final.merge(bot_counts, on='company_id', how='left')
    final['bot_count'] = final['bot_count'].fillna(0).astype(int)
else:
    final['has_bot'] = False
    final['has_prod_channel'] = False
    final['bot_count'] = 0

print(f"Bot columns added")

# --- Add credit wallet info ---
if credit_wallet is not None:
    wallet = credit_wallet.copy()
    wallet['company_id'] = pd.to_numeric(wallet['company_id'], errors='coerce')
    
    used_companies = wallet[wallet['total_used'] > 0]['company_id'].unique()
    final['used_conversations'] = final['company_id'].isin(used_companies)
    
    exceeded_companies = wallet[wallet['exceeded_free_tier'] == 1]['company_id'].unique()
    final['exceeded_free_tier'] = final['company_id'].isin(exceeded_companies)
else:
    final['used_conversations'] = False
    final['exceeded_free_tier'] = False

print(f"Wallet columns added")

# --- Add payment info ---
if stripe_invoices is not None:
    invoices = stripe_invoices.copy()
    invoices['company_id'] = pd.to_numeric(invoices['company_id'], errors='coerce')
    
    paid_invoices = invoices[invoices['amount_paid'] > 0]
    paid_companies = paid_invoices['company_id'].unique()
    final['actually_paid'] = final['company_id'].isin(paid_companies)
    
    # Total paid
    paid_summary = paid_invoices.groupby('company_id')['amount_paid'].sum().reset_index()
    paid_summary.columns = ['company_id', 'total_paid']
    final = final.merge(paid_summary, on='company_id', how='left')
    final['total_paid'] = final['total_paid'].fillna(0)
else:
    final['actually_paid'] = False
    final['total_paid'] = 0

print(f"Payment columns added")


In [ ]:
# --- Add template usage info ---
if template_usage is not None:
    tu = template_usage.copy()
    tu['company_id'] = pd.to_numeric(tu['company_id'], errors='coerce')
    
    # Aggregate by company_id (just in case there are duplicates)
    tu_agg = tu.groupby('company_id').agg({
        'total_events': 'sum',
        'created_templates': 'sum',
        'updated_templates': 'sum',
        'deleted_templates': 'sum'
    }).reset_index()
    
    final = final.merge(tu_agg, on='company_id', how='left')
    
    # Fill NaNs
    final['created_templates'] = final['created_templates'].fillna(0).astype(int)
    final['total_template_events'] = final['total_events'].fillna(0).astype(int)
    final['has_template_usage'] = final['created_templates'] > 0
    
    # Clean up duplicate total_events if any
    if 'total_events' in final.columns:
        final = final.drop(columns=['total_events'])
        
    print(f"Template usage columns added")
else:
    final['created_templates'] = 0
    final['total_template_events'] = 0
    final['has_template_usage'] = False
    print("No template usage data available")


In [ ]:
# --- Add session/retention info ---
if 'user_sessions' in dir() and user_sessions is not None:
    sessions = user_sessions.copy()
    sessions['company_id'] = pd.to_numeric(sessions['company_id'], errors='coerce')
    
    # Merge session data
    session_cols = ['company_id', 'first_session', 'last_session', 'days_active', 'total_sessions', 'user_count']
    session_cols = [c for c in session_cols if c in sessions.columns]
    
    final = final.merge(sessions[session_cols], on='company_id', how='left')
    
    # Merge session duration if available
    if 'sessions_duration' in dir() and sessions_duration is not None:
        final = final.merge(sessions_duration, on='company_id', how='left')
        final['total_time_minutes'] = final['total_time_minutes'].fillna(0)
        final['avg_session_minutes'] = final['avg_session_minutes'].fillna(0)
    
    # Calculate retention metrics
    final['signup_date'] = final['created_at'].dt.normalize()
    
    # Make sure session dates are tz-naive
    if final['last_session'].dt.tz is not None:
        final['last_session'] = final['last_session'].dt.tz_convert(None)
    if final['first_session'].dt.tz is not None:
        final['first_session'] = final['first_session'].dt.tz_convert(None)
    
    final['last_session_date'] = final['last_session'].dt.normalize()
    final['first_session_date'] = final['first_session'].dt.normalize()
    
    # Days to last activity (key retention metric)
    final['days_to_last_activity'] = (final['last_session_date'] - final['signup_date']).dt.days
    
    # Days since signup
    today = pd.Timestamp.now().normalize()
    final['days_since_signup'] = (today - final['signup_date']).dt.days
    
    # Pre-calculate retention flags for Streamlit
    # "Retained at Week N" = last activity was at least N*7 days after signup
    final['retained_day1'] = final['days_to_last_activity'] >= 1
    final['retained_week1'] = final['days_to_last_activity'] >= 7
    final['retained_week2'] = final['days_to_last_activity'] >= 14
    final['retained_week3'] = final['days_to_last_activity'] >= 21
    final['retained_week4'] = final['days_to_last_activity'] >= 28
    final['retained_week5'] = final['days_to_last_activity'] >= 35
    final['retained_week6'] = final['days_to_last_activity'] >= 42
    final['retained_week7'] = final['days_to_last_activity'] >= 49
    final['retained_week8'] = final['days_to_last_activity'] >= 56
    
    # Fill NaN retention flags with False (no session data = not retained)
    for col in ['retained_day1', 'retained_week1', 'retained_week2', 'retained_week3', 'retained_week4', 'retained_week5', 'retained_week6', 'retained_week7', 'retained_week8']:
        final[col] = final[col].fillna(False)
    
    # Days active as quality metric
    final['days_active'] = final['days_active'].fillna(0).astype(int)
    final['total_sessions'] = final['total_sessions'].fillna(0).astype(int)
    
    print(f"Session/retention columns added")
else:
    final['days_active'] = 0
    final['total_sessions'] = 0
    final['retained_day1'] = False
    final['retained_week1'] = False
    final['retained_week2'] = False
    final['retained_week3'] = False
    final['retained_week4'] = False
    final['retained_week5'] = False
    final['retained_week6'] = False
    final['retained_week7'] = False
    final['retained_week8'] = False
    print("No session data available")

print(f"\nFinal dataframe: {len(final)} rows, {len(final.columns)} columns")

    # Merge nodes usage if available
    if 'nodes_usage' in dir() and nodes_usage is not None:
        # Aggregate total nodes per company
        nodes_agg = nodes_usage.groupby('company_id')['nodes_created'].sum().reset_index(name='total_nodes_created')
        final = final.merge(nodes_agg, on='company_id', how='left')
        final['total_nodes_created'] = final['total_nodes_created'].fillna(0).astype(int)
    else:
        final['total_nodes_created'] = 0


In [ ]:
# Filter out internal Jelou accounts
before = len(final)

# Remove @jelou.ai emails
if 'email' in final.columns:
    final = final[~final['email'].str.contains('@jelou.ai', case=False, na=False)]

# Remove jelou-related slugs
if 'slug' in final.columns:
    final = final[~final['slug'].str.contains('jelou', case=False, na=False)]

# Remove test companies from test_companies.xlsx
test_companies_path = 'data/test_companies.xlsx'
if os.path.exists(test_companies_path):
    try:
        test_companies_df = pd.read_excel(test_companies_path)
        first_col = test_companies_df.columns[0]
        test_company_names = test_companies_df[first_col].dropna().str.strip().str.lower().tolist()
        if 'company_name' in final.columns:
            before_test = len(final)
            final = final[~final['company_name'].str.strip().str.lower().isin(test_company_names)]
            print(f"Filtered out {before_test - len(final)} test companies from test_companies.xlsx")
    except Exception as e:
        print(f"Warning: Could not load test_companies.xlsx: {e}")

after = len(final)
print(f"Removed {before - after} internal Jelou accounts")
print(f"Final clean dataset: {len(final)} companies")


In [ ]:
asd = [2829,2831,2832,2834,2835,2839,2840,2841,2842,2844,2845,2848,2849,2851,2853,2854,2855,2856,2858,2859,2860,2861,2862,2863,2864,2865,2866,2867,2868,2869,2870,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880,2882,2883,2884,2885,2887,2888,2889,2890,2891,2892,2893,2894,2895,2896,2897,2898,2899,2901,2902,2903,2904,2905,2906,2908,2909,2910,2911,2912,2913,2914,2915,2916,2917,2918,2919,2920,2921,2922,2923,2924,2925,2926,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2952,2953,2954,2955,2956,2957,2959,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2972,2973,2975,2976,2977,2978,2979,2980,2981,2982,2984,2985,2986,2987,2988,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,3012,3013,3014,3015,3016,3017,3018,3019,3020,3021,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031,3032,3033,3034,3035,3036,3037,3038,3040,3041,3042,3043,3044,3045,3046,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3071,3073,3075,3076,3077,3079,3082,3085,3088,3089,3090,3091,3092,3093,3094,3095,3096,3097,3098,3099,3100,3101,3102,3103,3104,3105,3106,3107,3108,3110,3111,3112,3113,3114,3115,3116,3117,3118,3119,3120,3121,3122,3124,3125,3126,3127,3128,3129,3130,3131,3132,3133,3134,3135,3137,3138,3139,3140,3141,3142,3143,3144,3145,3146,3147,3148,3149,3150,3151,3152,3153,3155,3156,3157,3158,3159,3160,3161,3162,3163,3165,3167,3168,3169,3170,3171,3172,3173,3174,3175,3176,3178,3179,3180,3181,3182,3183,3184,3185,3186,3187,3188,3189,3192,3193,3194,3195,3196,3199,3200,3201,3202,3203,3204,3205,3206,3208,3209,3210,3211,3212,3213,3214,3215,3216,3217,3218,3219,3220,3221,3223,3224,3225,3226,3227,3228,3229,3230,3231,3232,3233,3234,3235,3236,3237,3238,3239,3240,3241,3242,3243,3244,3245,3246,3247,3248,3249,3250,3251,3252,3253,3259,3260,3262,3263,3264,3265,3266,3267,3268,3269,3270,3271,3272,3273,3274,3275,3276,3277,3278,3279,3280,3281,3282,3283,3284,3285,3286,3287,3288,3289,3290,3291,3292,3293,3294,3295,3296,3297,3298,3299,3300,3301,3302,3303,3304,3305,3306,3307,3308]
len(asd)

In [ ]:
# Final summary and save
print("=" * 60)
print("FINAL ANALYSIS SUMMARY")
print("=" * 60)

total = len(final)
ss_total = len(final[final['plan'] == 'SELF_SERVICE'])

print(f"\nTotal companies: {total}")
print(f"  - SELF_SERVICE: {ss_total}")
print(f"  - ENTERPRISE: {len(final[final['plan'] == 'ENTERPRISE'])}")
print(f"  - Other: {total - ss_total - len(final[final['plan'] == 'ENTERPRISE'])}")

print(f"\n=== FUNNEL (SELF_SERVICE) ===")
ss = final[final['plan'] == 'SELF_SERVICE']
print(f"1. Signups: {len(ss)}")
print(f"2. Created Bot: {ss['has_bot'].sum()} ({ss['has_bot'].sum()/len(ss)*100:.1f}%)")
print(f"3. Production Channel: {ss['has_prod_channel'].sum()} ({ss['has_prod_channel'].sum()/len(ss)*100:.1f}%)")
print(f"4. Used Conversations: {ss['used_conversations'].sum()} ({ss['used_conversations'].sum()/len(ss)*100:.1f}%)")
print(f"5. Exceeded Free Tier: {ss['exceeded_free_tier'].sum()} ({ss['exceeded_free_tier'].sum()/len(ss)*100:.1f}%)")
print(f"6. Actually Paid: {ss['actually_paid'].sum()} ({ss['actually_paid'].sum()/len(ss)*100:.1f}%)")

print(f"\n=== RETENTION (SELF_SERVICE) ===")
# Only count signups old enough
ss_7days = ss[ss['days_since_signup'] >= 7]
ss_14days = ss[ss['days_since_signup'] >= 14]
ss_21days = ss[ss['days_since_signup'] >= 21]
ss_28days = ss[ss['days_since_signup'] >= 28]
ss_35days = ss[ss['days_since_signup'] >= 35]
ss_42days = ss[ss['days_since_signup'] >= 42]
ss_49days = ss[ss['days_since_signup'] >= 49]
ss_56days = ss[ss['days_since_signup'] >= 56]
ss_63days = ss[ss['days_since_signup'] >= 63]

print(f"Day 1:  {ss_7days['retained_day1'].sum()}/{len(ss_7days)} = {ss_7days['retained_day1'].sum()/len(ss_7days)*100:.1f}%" if len(ss_7days) > 0 else "N/A")
print(f"Week 1: {ss_14days['retained_week1'].sum()}/{len(ss_14days)} = {ss_14days['retained_week1'].sum()/len(ss_14days)*100:.1f}%" if len(ss_14days) > 0 else "N/A")
print(f"Week 2: {ss_21days['retained_week2'].sum()}/{len(ss_21days)} = {ss_21days['retained_week2'].sum()/len(ss_21days)*100:.1f}%" if len(ss_21days) > 0 else "N/A")
print(f"Week 3: {ss_28days['retained_week3'].sum()}/{len(ss_28days)} = {ss_28days['retained_week3'].sum()/len(ss_28days)*100:.1f}%" if len(ss_28days) > 0 else "N/A")
print(f"Week 4: {ss_28days['retained_week4'].sum()}/{len(ss_28days)} = {ss_28days['retained_week4'].sum()/len(ss_28days)*100:.1f}%" if len(ss_28days) > 0 else "N/A")
print(f"Week 5: {ss_35days['retained_week5'].sum()}/{len(ss_35days)} = {ss_35days['retained_week5'].sum()/len(ss_35days)*100:.1f}%" if len(ss_35days) > 0 else "N/A")
print(f"Week 6: {ss_42days['retained_week6'].sum()}/{len(ss_42days)} = {ss_42days['retained_week6'].sum()/len(ss_42days)*100:.1f}%" if len(ss_42days) > 0 else "N/A")
print(f"Week 7: {ss_49days['retained_week7'].sum()}/{len(ss_49days)} = {ss_49days['retained_week7'].sum()/len(ss_49days)*100:.1f}%" if len(ss_49days) > 0 else "N/A")
print(f"Week 8: {ss_56days['retained_week8'].sum()}/{len(ss_56days)} = {ss_56days['retained_week8'].sum()/len(ss_56days)*100:.1f}%" if len(ss_56days) > 0 else "N/A")




print(f"\n=== DAYS ACTIVE DISTRIBUTION (SELF_SERVICE) ===")
with_activity = ss[ss['days_active'] > 0]
print(f"Companies with any session: {len(with_activity)}/{len(ss)} ({len(with_activity)/len(ss)*100:.1f}%)")
if len(with_activity) > 0:
    print(f"  1 day only: {(with_activity['days_active'] == 1).sum()}")
    print(f"  2-7 days: {((with_activity['days_active'] >= 2) & (with_activity['days_active'] <= 7)).sum()}")
    print(f"  8-14 days: {((with_activity['days_active'] >= 8) & (with_activity['days_active'] <= 14)).sum()}")
    print(f"  15-28 days: {((with_activity['days_active'] >= 15) & (with_activity['days_active'] <= 28)).sum()}")
    print(f"  29+ days: {(with_activity['days_active'] >= 29) & (with_activity['days_active'] <= 35).sum()}")
    print(f"  35+ days: {(with_activity['days_active'] >= 35) & (with_activity['days_active'] <= 42).sum()}")
    print(f"  42+ days: {(with_activity['days_active'] >= 42) & (with_activity['days_active'] <= 49).sum()}")
    print(f"  49+ days: {(with_activity['days_active'] >= 49) & (with_activity['days_active'] <= 56).sum()}")
    print(f"  56+ days: {(with_activity['days_active'] >= 56) & (with_activity['days_active'] <= 63).sum()}")
    print(f"  63+ days: {(with_activity['days_active'] >= 63).sum()}")

In [ ]:
# Save to CSV for Streamlit
# Select columns to save (avoid duplicates and unnecessary columns)
save_cols = [
    # Core info
    'company_id', 'company_name', 'slug', 'type', 'plan', 'email', 
    'in_production', 'state', 'environment', 'country', 'timezone',
    'created_at', 'updated_at',
    
    # Subscription flags
    'has_subscription', 'has_active', 'has_trialing',
    'has_brain_studio', 'brain_active',
    'has_connect', 'connect_active', 'connect_trialing', 'has_template_usage', 'created_templates', 'total_template_events',
    
    # Bot flags
    'has_bot', 'has_prod_channel', 'bot_count',
    
    # Usage/payment flags
    'used_conversations', 'exceeded_free_tier', 'actually_paid', 'total_paid',
    
    # Session/retention data
    'first_session', 'last_session', 'days_active', 'total_sessions',
    'days_to_last_activity', 'days_since_signup', 'total_time_minutes', 'avg_session_minutes', 'total_nodes_created',
    'retained_day1', 'retained_week1', 'retained_week2', 'retained_week3', 'retained_week4', 'retained_week5', 'retained_week6', 'retained_week7', 'retained_week8'
]

# Only include columns that exist
save_cols = [c for c in save_cols if c in final.columns]

# Save
final[save_cols].to_csv('data/analysis_combined.csv', index=False)
print(f"\n✅ Saved: data/analysis_combined.csv")
print(f"   {len(final)} rows, {len(save_cols)} columns")
print(f"\nColumns saved:")
for col in save_cols:
    print(f"  - {col}")